### Logistic Regression ###

In [1]:
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification
from pyspark.sql import functions as fn, Row
from pyspark import sql
import re
import time

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
flights_lr_clean = spark.read.parquet('flights_transformed.parquet')

In [3]:
# Make a training, validation, and test split #
training_df, validation_df, testing_df = flights_lr_clean.randomSplit([0.6, 0.3, 0.1], seed=0)

In [4]:
# Build a logistic regression model (default params) using the training data #
lr_model = classification.LogisticRegression(featuresCol='features',labelCol='DEPARTURE_DELAY_index').fit(training_df)

In [12]:
# Transform it against the validation data #
lr_df = lr_model.transform(validation_df).select('prediction','DEPARTURE_DELAY_index').toPandas()

In [13]:
# Show the results #
evaluation_lr_df = pd.concat([lr_df['prediction'],lr_df['DEPARTURE_DELAY_index']], 
                          axis = 1, keys=['prediction','label'])
crosstab_lr_df = pd.crosstab(evaluation_lr_df.prediction, evaluation_lr_df.label)
display(crosstab_lr_df)
print("Accuracy: {:,.3f}%" .format((crosstab_lr_df[0][0] + crosstab_lr_df[1][1]) / crosstab_lr_df.values.sum() * 100))

label,0.0,1.0
prediction,,
0.0,12198,5612
1.0,4298,4328


Accuracy: 62.513%
